In [67]:
from lxml import html
from json import dump,loads
from requests import get
import json
from re import sub
from dateutil import parser as dateparser
from time import sleep


In [68]:
amazon_url  = ('https://www.amazon.com/product-reviews/{0}/ref=cm_cr_getr_d_paging_btm_1?ie=UTF8&reviewerType=all_reviews&pageNumber=1'
    .format("B01DFKC2SO")) #asin is unique amazon id

headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.13 (KHTML, like Gecko) Chrome/9.0.597.0 Safari/534.13'}

In [69]:
response = get(amazon_url, headers = headers, verify=False, timeout=30)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [70]:
response

<Response [200]>

In [71]:
cleaned_response = response.text.replace('\x00', '')
cleaned_response

'<!doctype html><html lang="en-us" class="a-no-js" data-19ax5a9jf="dingo"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8">\n<script type=\'text/javascript\'>var ue_t0=ue_t0||+new Date();</script><!-- sp:feature:cs-optimization -->\n<meta http-equiv=\'x-dns-prefetch-control\' content=\'on\'><link rel=\'dns-prefetch\' href=\'//images-na.ssl-images-amazon.com\'><link rel=\'dns-prefetch\' href=\'//m.media-amazon.com\'><link rel=\'dns-prefetch\' href=\'//completion.amazon.com\'><script type=\'text/javascript\'>\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],

In [54]:
parser = html.fromstring(cleaned_response)
parser

<Element html at 0x104bb3db8>

In [56]:
XPATH_AGGREGATE = '//span[@id="acrCustomerReviewText"]'
XPATH_REVIEW_SECTION_1 = '//div[contains(@id,"reviews-summary")]'
XPATH_REVIEW_SECTION_2 = '//div[@data-hook="review"]'
XPATH_AGGREGATE_RATING = '//table[@id="histogramTable"]//tr'
XPATH_PRODUCT_NAME = './/h1[@class="a-size-large a-text-ellipsis"]//text()'
XPATH_PRODUCT_PRICE = '//span[@id="priceblock_ourprice"]/text()'

raw_product_price = parser.xpath(XPATH_PRODUCT_PRICE)
raw_product_name = parser.xpath(XPATH_PRODUCT_NAME)
total_ratings  = parser.xpath(XPATH_AGGREGATE_RATING)
reviews = parser.xpath(XPATH_REVIEW_SECTION_1)

product_price = ''.join(raw_product_price).replace(',', '')
product_name = ''.join(raw_product_name).strip()
if not reviews:
            reviews = parser.xpath(XPATH_REVIEW_SECTION_2)
ratings_dict = {}
reviews_list = []

In [57]:
XPATH_RATING  = './/i[@data-hook="review-star-rating"]//text()'
XPATH_REVIEW_HEADER = './/a[@data-hook="review-title"]//text()'
XPATH_REVIEW_POSTED_DATE = './/span[@data-hook="review-date"]//text()'
XPATH_REVIEW_TEXT_1 = './/div[@class="a-row a-spacing-small review-data"]//text()'
XPATH_REVIEW_TEXT_2 = './/div//span[@data-action="columnbalancing-showfullreview"]/@data-columnbalancing-showfullreview'
XPATH_REVIEW_COMMENTS = './/span[@data-hook="review-comment"]//text()'
XPATH_AUTHOR = './/span[contains(@class,"profile-name")]//text()'
XPATH_REVIEW_TEXT_3 = './/div[contains(@id,"dpReviews")]/div/text()'

In [60]:
raw_product_name

[]

In [51]:
parser.xpath(XPATH_REVIEW_POSTED_DATE)

[]

In [53]:
parser.xpath(XPATH_REVIEW_SECTION_1)

[]

In [10]:
raw_review_text1 = reviews[0].xpath(XPATH_REVIEW_TEXT_1)
raw_review_text1

["Because of this little hockey puck I've been on a mission to automate my home!",
 'I knew one day we would eventually live in those "Back to the Future" type homes, but I never realized it would be today!',
 "I bought this bundled with a TP-Link smart plug, just to try out. Amazon had a GREAT deal on it, so I figured why not...if I didn't like it, wasn't a big loss.",
 'As it turns out, my whole family and I ended up having fun with the Echo Dot. Asking Alexa every thing from how the weather is to how big the universe is. My favorite is probably asking Alexa to play a certain song or playlist out of the Prime music library...',
 'I hooked the TP-Link smart plug up to my Christmas tree lights, and the kids have a blast asking Alexa to turn the tree light on and off.',
 "Yup, because of this Dot, in the past 3 weeks I've ordered an Echo Plus bundled with a Hue Bulb that ran with a special for a TP-Link smart plug (now i have two) at a ridiculously low price, a Fire stick, a Fire tablet

In [11]:
review_text = ' '.join(' '.join(raw_review_text1).split())
review_text

'Because of this little hockey puck I\'ve been on a mission to automate my home! I knew one day we would eventually live in those "Back to the Future" type homes, but I never realized it would be today! I bought this bundled with a TP-Link smart plug, just to try out. Amazon had a GREAT deal on it, so I figured why not...if I didn\'t like it, wasn\'t a big loss. As it turns out, my whole family and I ended up having fun with the Echo Dot. Asking Alexa every thing from how the weather is to how big the universe is. My favorite is probably asking Alexa to play a certain song or playlist out of the Prime music library... I hooked the TP-Link smart plug up to my Christmas tree lights, and the kids have a blast asking Alexa to turn the tree light on and off. Yup, because of this Dot, in the past 3 weeks I\'ve ordered an Echo Plus bundled with a Hue Bulb that ran with a special for a TP-Link smart plug (now i have two) at a ridiculously low price, a Fire stick, a Fire tablet for the baby, an

In [12]:
XPATH_HELPFUL_VOTE = './/span[contains(@data-hook,"helpful-vote-statement")]'
XPATH_HELPFUL_VOTE_DIV = './/div[@class="a-row a-spacing-small"]//text()'
TEST_XPATH = './/div[@class="a-size-base a-color-tertiary cr-vote-text"]//text()'
raw_comments = reviews[0].xpath(XPATH_HELPFUL_VOTE_DIV)[0].split(' ')[0]
raw_comments



'One'

In [13]:
XPATH_REVIEW_COMMENTS = './/span[@class="a-size-base"]//text()'
raw_review_count = reviews[0].xpath(XPATH_REVIEW_COMMENTS)
raw_review_count[0][0]

'C'

In [14]:
XPATH_PRICE = './/span[@class="a-color-price arp-price"]'

In [15]:
parser.xpath(XPATH_PRICE)[0].text_content().split(' ')[-1]

'$44.99'

In [16]:
XPATH_NAME = '//h1//div[@data-hook="product-link"]'


In [17]:
parser.xpath(XPATH_NAME)

[]